In [ ]:
# import the needed packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

import matplotlib.image as img

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Conv2D

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from fast_ml.model_development import train_valid_test_split

In [ ]:
# Caminhos de treino e validaçção

# Obs - no diretório dados4 os dados estão separados em 75% treino 15% validação e 10% teste
path_train = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/train/'
path_val = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/val/'

batch_size = 20

In [ ]:
# Gerando os dados a partir das imagens
train_datagen = ImageDataGenerator(    
    rotation_range = 90,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range=0.05,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode = 'reflect',
    samplewise_center = True,
    samplewise_std_normalization = True
)

val_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )

In [ ]:
# dados de treino
train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size = (256, 256),
    batch_size= batch_size,
    shuffle=True,
    class_mode='categorical' 
)

In [ ]:
y_true = train_generator.classes
y_true

In [ ]:
#dados de validação
val_generator = val_datagen.flow_from_directory(
    path_val,
    target_size = (256, 256),
    batch_size= batch_size,  
    shuffle=True,
    class_mode='categorical'     
)

In [ ]:
val_generator.classes

In [ ]:
early_stoppping = EarlyStopping(monitor='val_loss',
                                patience=15,
                                restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.1,
                                         patience=15)

lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate = 0.01, 
    first_decay_steps = 500, 
    t_mul=2.4, 
    m_mul=1.3, 
    alpha=0.001, 
    name=None
)

xception = Xception(include_top = False,input_shape = (256,256,3))
input_to_model = xception.input
#turn off training
xception.trainable = False

x = Flatten()(xception.output)
x = Dense(64,activation = 'relu')(x)
output_to_model = Dense(2,activation = 'softmax')(x)
model = Model(inputs = input_to_model,outputs = output_to_model)
model.summary()

In [ ]:
model.compile(optimizer = keras.optimizers.SGD(learning_rate=lr_schedule),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy',Recall(),'AUC'])

In [ ]:
history = model.fit(x = train_generator,
                    batch_size=len(train_generator),
                    epochs=25,
                    callbacks = [early_stoppping,reduce_lr_on_plateau],
                    validation_data = val_generator)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for auc
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#dados de teste
path_test = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/U-net/test/'
test_generator =  test_datagen.flow_from_directory(
    path_test,
    target_size = (256, 256),
    shuffle = False,
    batch_size= batch_size,
    class_mode='categorical' 
    
    )

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print(classification_report(test_generator.classes, y_pred))

In [ ]:
class_names = ['Benign','Malignant']
report = classification_report(test_generator.classes, y_pred,target_names=class_names, output_dict=True)
df = pd.DataFrame(report).transpose()
df_ = df[['precision', 'recall','f1-score']]
df_ = df_.loc[["Benign", "Malignant","macro avg"]]
df_.to_csv('U-net_Xception.csv')
df_.head(10)

In [ ]:
cm = confusion_matrix(test_generator.classes, y_pred)
#cm = [[24,11],[1,34]]
plt.figure(figsize=(5,5))
x_axis_labels = ['Benign','Malignant'] # labels for x-axis
y_axis_labels = ['Benign','Malignant'] # labels for y-axis
ax = sns.heatmap(cm, annot=True, fmt="d",xticklabels=x_axis_labels, yticklabels=y_axis_labels)
sns.set(font_scale=1.8)
plt.title('Confusion matrix Xception')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')